In [48]:
import grammar as g
import tmp as t


ModuleNotFoundError: No module named 'utilities'

In [32]:
gramm = g.Grammar('/home/alexserra98/uni/prog_deep/Genetics_ANN/GA_for_param_optim/cnn.grammar.txt')


In [33]:
for i in gramm.grammar:
    print(f'{i} : {gramm.grammar[i]}')

features : [[('convolution', True), ('pooling', True)], [('convolution', True)], [('pooling', True)], [('pooling', True)], [('dropout', True)], [('batch-norm', True)]]
convolution : [[('layer:conv', False), ('[num-filters,int,1,32,256]', False), ('[filter-shape,int,1,2,5]', False), ('[stride,int,1,1,3]', False), ('padding', True), ('activation-function', True), ('bias', True)]]
batch-norm : [[('layer:batch-norm', False)]]
pooling : [[('pool-type', True), ('[kernel-size,int,1,2,5]', False), ('[stride,int,1,1,3]', False), ('padding', True)]]
pool-type : [[('layer:pool-avg', False)], [('layer:pool-max', False)]]
padding : [[('padding:same', False)], [('padding:valid', False)]]
dropout : [[('layer:dropout', False), ('[rate,float,1,0,0.7]', False)]]
classification : [[('fully-connected', True)], [('dropout', True)]]
fully-connected : [[('layer:fc', False), ('activation-function', True), ('[num-units,int,1,128,2048]', False), ('bias', True)]]
activation-function : [[('act:linear', False)], [

In [42]:
gramm.grammar['convolution'][0]

[('layer:conv', False),
 ('[num-filters,int,1,32,256]', False),
 ('[filter-shape,int,1,2,5]', False),
 ('[stride,int,1,1,3]', False),
 ('padding', True),
 ('activation-function', True),
 ('bias', True)]

In [43]:
prova = gramm.initialise('features')

In [44]:
for i in prova:
    print(f'{i} : {prova[i]}')

features : [{'ge': 0, 'ga': {}}]
convolution : [{'ge': 0, 'ga': {'num-filters': ('int', 32.0, 256.0, [165]), 'filter-shape': ('int', 2.0, 5.0, [3]), 'stride': ('int', 1.0, 3.0, [2])}}]
padding : [{'ge': 0, 'ga': {}}, {'ge': 0, 'ga': {}}]
activation-function : [{'ge': 1, 'ga': {}}]
bias : [{'ge': 0, 'ga': {}}]
pooling : [{'ge': 0, 'ga': {'kernel-size': ('int', 2.0, 5.0, [4]), 'stride': ('int', 1.0, 3.0, [2])}}]
pool-type : [{'ge': 1, 'ga': {}}]


In [21]:
type(prova['features'])

list

In [46]:
feat_layers = gramm.decode('features', prova)

In [47]:
layer_decoding = t.Evaluator()
layer_decoding.get_layers(feat_layers) 

NameError: name 't' is not defined